In [1]:
import pandas as pd
import numpy as np

In [2]:
#Load data
endpoint_profiles_all = pd.read_csv("./inputData/endpoint_profiles_all.csv", index_col=None)

# Combine both hemispheres
endpoint_profiles_all['t1t2w_GM'] = endpoint_profiles_all[['t1wt2wValues_LH', 't1wt2wValues_RH']].sum(axis=1, skipna=True)

# Compute weighted average T1w/T2w per subject/session/tract
WeightedAverageT1T2wSurface = (
    endpoint_profiles_all
    .groupby(['subjectID', 'sessionID', 'tractID'], as_index=False)
    .apply(lambda x: pd.Series({
        'WeAvGMT1wT2w': np.average(x['t1t2w_GM'], weights=x['endpointdensity'])
    }))
    .reset_index(drop=True)
)

# Result
WeightedAverageT1T2wSurface.to_csv('./inputData/GreyMatterT1wT2w_WeightedAverage.csv')

In [3]:
#Load and edit white matter tract profiles
tract_profilesCleaned = pd.read_csv('./inputData/TractProfilesCleaned.csv', index_col=None)

#Delete nodes 0-4 and 95-100 to prevent partial volume effects
nodes_to_remove = list(range(0, 5)) + list(range(95, 100))
tract_profilesCleanedCut = tract_profilesCleaned[~tract_profilesCleaned['nodeID'].isin(nodes_to_remove)]
tract_profilesCleanedCut

#Take mean of 100 nodes
TractProfilesMeanCut=tract_profilesCleanedCut.groupby(['subjectID', 'sessionID', 'tractID'], as_index=False).mean(['t1wt2w'])
TractProfilesMeanCut.to_csv('./inputData/TractProfilesMeanCut.csv')